In [19]:
from binance.client import Client
from binance.exceptions import BinanceAPIException, BinanceRequestException
import pandas as pd

In [20]:
client = Client("0ppfXZtnEyP5tINDcKvVSh5XLwPNHChrS7MTQ44MgS7zVXEeO1QqpICaHBr94bGa", "LyAdCy4nMkwSOGVf7lAdwjXnI7ijPoLn6WiPOMhFwn8Lc8hYvn4VtKh3755AW6aA")

In [85]:

symbol = "BTCUSDT" 
interval = Client.KLINE_INTERVAL_1HOUR # intervalo de tiempo para las velas 
limit = 2 # limite de velas
klines = client.get_klines(symbol=symbol,interval=interval,limit=limit)

barss = pd.DataFrame(klines, columns=[
    'Open time', 'Open', 'High', 'Low', 'Close', 'Volume',
    'Close time', 'Quote asset volume', 'Number of trades',
    'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'
])

barss['Open time'] = pd.to_datetime(barss['Open time'], unit='ms') # formatea la fecha
bars = barss.iloc[[0]] # accede a la primera fila del dataframe 'penultima vela' 
bars

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,2024-10-02 04:00:00,61750.46000000,61750.47000000,61440.00000000,61495.79000000,704.31237000,1727845199999,43359409.11232200,112478,285.14662000,17555124.40581380,0
